/Users/tedwu/anaconda3/envs/CS5344/lib/python3.11/site-packages/tensorflow


In [1]:
import sparknlp
print(sparknlp.version())

5.1.2


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, FloatType, ArrayType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

spark = SparkSession.builder \
    .appName(
        "SentimentAnalysisRoBERTaRF"
    ).config(
        "spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.0"
    ).config(
        "spark.executor.memory", "16g"
    ).config(
        "spark.driver.memory", "32g"
    ).getOrCreate()

print("Spark NLP version: ", sparknlp.version())

:: loading settings :: url = jar:file:/Users/tedwu/anaconda3/envs/CS5344/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tedwu/.ivy2/cache
The jars for the packages stored in: /Users/tedwu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0fbf36a2-6aae-4f83-90e3-d3e61342df2e;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lom

Spark NLP version:  4.2.0


In [16]:
df = spark.read.csv("../ggg_sg.csv", header=True, inferSchema=True, multiLine=True, escape='"')

df = df.filter(df.ContextualText.isNotNull())
df = df.filter(df.DocTone.isNotNull())

df = df.withColumn("DocTone", df["DocTone"].cast(FloatType()))

def sentiment_label(score):
    if score > 1.9910:
        return 2
    elif score < -2.0202:
        return 0
    else:
        return 1

sentiment_udf = udf(sentiment_label, IntegerType())
df = df.withColumn("label", sentiment_udf(col("DocTone")))

label_counts = df.groupBy("label").count().orderBy("label")
label_counts.show()

+-----+-------+
|label|  count|
+-----+-------+
|    0|2297409|
|    1|4586529|
|    2|2301367|
+-----+-------+



In [21]:
# Create a Spark NLP pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("ContextualText") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

# Use RoBERTa for embeddings
roberta_embeddings = RoBertaEmbeddings.pretrained("roberta_base", "en") \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("embeddings") \
    .setPoolingStrategy("mean")

# Finisher will output the embeddings to a list
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["embeddings"]) \
    .setOutputCols(["finished_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

roberta_base download started this may take some time.
Approximate size to download 284.8 MB
[ / ]roberta_base download started this may take some time.
Approximate size to download 284.8 MB
[ — ]Download done! Loading the resource.
[ | ]
An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: java.lang.NoClassDefFoundError: Could not initialize class org.tensorflow.Graph
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.readGraph(TensorflowWrapper.scala:415)
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.unpackWithoutBundle(TensorflowWrapper.scala:330)
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.read(TensorflowWrapper.scala:484)
	at com.johnsnowlabs.ml.tensorflow.ReadTensorflowModel.readTensorflowModel(TensorflowSerializeModel.scala:154)
	at com.johnsnowlabs.ml.tensorflow.ReadTensorflowModel.readTensorflowModel$(TensorflowSerializeModel.scala:123)
	at com.johnsnowlabs.nlp.embeddings.RoBertaEmbeddings$.readTensorf

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: java.lang.NoClassDefFoundError: Could not initialize class org.tensorflow.Graph
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.readGraph(TensorflowWrapper.scala:415)
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.unpackWithoutBundle(TensorflowWrapper.scala:330)
	at com.johnsnowlabs.ml.tensorflow.TensorflowWrapper$.read(TensorflowWrapper.scala:484)
	at com.johnsnowlabs.ml.tensorflow.ReadTensorflowModel.readTensorflowModel(TensorflowSerializeModel.scala:154)
	at com.johnsnowlabs.ml.tensorflow.ReadTensorflowModel.readTensorflowModel$(TensorflowSerializeModel.scala:123)
	at com.johnsnowlabs.nlp.embeddings.RoBertaEmbeddings$.readTensorflowModel(RoBertaEmbeddings.scala:488)
	at com.johnsnowlabs.nlp.embeddings.ReadRobertaTensorflowModel.readTensorflow(RoBertaEmbeddings.scala:427)
	at com.johnsnowlabs.nlp.embeddings.ReadRobertaTensorflowModel.readTensorflow$(

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Create a feature vector
vector_assembler = VectorAssembler(inputCols=["finished_embeddings"], outputCol="features")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    stopwords_cleaner,
    roberta_embeddings,
    embeddings_finisher,
    vector_assembler
])

(trainingData, testData) = df.randomSplit([0.8, 0.2], seed=42)

pipelineModel = pipeline.fit(trainingData)
trainingData = pipelineModel.transform(trainingData)
testData = pipelineModel.transform(testData)

rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=50, maxDepth=10)

rfModel = rf.fit(trainingData)


In [ ]:
predictions = rfModel.transform(testData)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % accuracy)

precision_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

precision = precision_evaluator.evaluate(predictions)
recall = recall_evaluator.evaluate(predictions)
f1 = f1_evaluator.evaluate(predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

predictions.select("ContextualText", "label", "prediction").show(10, truncate=50)

spark.stop()